# NFL Full‑Stack Weekly Predictions

**Generated:** 2025-11-02T16:39:13

Two spaces after periods.  This notebook reproduces the full pipeline for all games in a slate using your CSV input.

## Build FILE

In [ ]:
%%python
import pandas as pd
import numpy as np
import requests, io, re
from datetime import datetime
from nfl_data_py import import_schedules
from bs4 import BeautifulSoup
from pathlib import Path

# ---------------------------------------------------------------------------
# 1. Determine current season/week
# ---------------------------------------------------------------------------
current_year = 2025
sched = import_schedules([current_year])
today = datetime.now().strftime("%Y-%m-%d")
this_week = sched.loc[sched["game_date"] <= today, "week"].max() + 1
games = sched.query("season == @current_year and week == @this_week")[["game_id","home_team","away_team","game_date","start_time"]].copy()
print(f"Detected current week = {this_week}")

# ---------------------------------------------------------------------------
# 2. Odds from ESPN public scoreboard
# ---------------------------------------------------------------------------
def get_odds():
    url = "https://site.web.api.espn.com/apis/v2/scoreboard/header?sport=football&league=nfl"
    r = requests.get(url).json()
    odds = []
    for ev in r.get("events", []):
        comp = ev.get("competitions", [{}])[0]
        if not comp.get("odds"): continue
        o = comp["odds"][0]
        fav_spread = None
        if "details" in o and "−" in o["details"]:
            try:
                fav_spread = float(o["details"].split("−")[-1])
            except Exception:
                fav_spread = None
        odds.append({
            "home_team": comp["competitors"][0]["team"]["abbreviation"],
            "away_team": comp["competitors"][1]["team"]["abbreviation"],
            "spread_fav": fav_spread,
            "total": o.get("overUnder"),
            "ml_fav": o.get("moneyLine"),
            "source": o.get("provider",{}).get("name","ESPN")
        })
    return pd.DataFrame(odds)

odds = get_odds()

# ---------------------------------------------------------------------------
# 3. Weather from Open-Meteo
# ---------------------------------------------------------------------------
city_lookup = {
    "GB": "Green Bay,US","MIA": "Miami,US","NYG": "East Rutherford,US","HOU": "Houston,US",
    "PIT": "Pittsburgh,US","CIN": "Cincinnati,US","DET": "Detroit,US","TEN": "Nashville,US",
    "NE": "Foxborough,US","LV": "Las Vegas,US","LAR": "Inglewood,US","BUF": "Buffalo,US",
    "WAS": "Landover,US","DAL": "Arlington,US","SEA": "Seattle,US","KC": "Kansas City,US",
    "NO": "New Orleans,US","ATL": "Atlanta,US","CHI": "Chicago,US","IND": "Indianapolis,US",
    "PHI": "Philadelphia,US","SF": "Santa Clara,US","MIN": "Minneapolis,US",
    "BAL": "Baltimore,US","CLE": "Cleveland,US","JAX": "Jacksonville,US",
    "NYJ": "East Rutherford,US","LA": "Inglewood,US","CAR": "Charlotte,US","ARI": "Glendale,US",
    "DEN": "Denver,US","TB": "Tampa,US"
}

def fetch_weather(team):
    city = city_lookup.get(team)
    if not city: return (np.nan,np.nan,np.nan)
    try:
        geo = requests.get(f"https://geocoding-api.open-meteo.com/v1/search?name={city}").json()
        lat, lon = geo["results"][0]["latitude"], geo["results"][0]["longitude"]
        w = requests.get(
            f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&hourly=temperature_2m,precipitation_probability,windspeed_10m&forecast_days=1"
        ).json()
        t = np.mean(w["hourly"]["temperature_2m"])
        p = np.mean(w["hourly"]["precipitation_probability"])
        wind = np.mean(w["hourly"]["windspeed_10m"])
        return (p, t, wind)
    except Exception:
        return (np.nan,np.nan,np.nan)

wx = [fetch_weather(t) for t in games["home_team"]]
games[["weather_precip","weather_temp_f","weather_breezy"]] = pd.DataFrame(wx)

# ---------------------------------------------------------------------------
# 4. EPA/play from RBSDM
# ---------------------------------------------------------------------------
def fetch_team_epa():
    url = "https://rbsdm.com/stats/nfl_team_stats.csv"
    r = requests.get(url)
    df = pd.read_csv(io.StringIO(r.text))
    df.columns = [c.lower() for c in df.columns]
    epa = df[["team","offense_epa_per_play","defense_epa_per_play"]].copy()
    epa["team"] = epa["team"].str.upper().str[:3]
    return epa

epa = fetch_team_epa()

# ---------------------------------------------------------------------------
# 5. Injury report from FantasyPros
# ---------------------------------------------------------------------------
def scrape_injuries():
    url = "https://www.fantasypros.com/nfl/injuries/"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    tables = soup.find_all("table")
    inj_data = []
    for table in tables:
        headers = [th.get_text(strip=True) for th in table.find("thead").find_all("th")]
        for tr in table.find("tbody").find_all("tr"):
            tds = [td.get_text(strip=True) for td in tr.find_all("td")]
            if len(tds) >= 3:
                team = tds[1].upper()[:3]
                status = tds[-1].lower()
                inj_data.append({"team": team, "status": status})
    df = pd.DataFrame(inj_data)
    counts = (
        df.groupby(["team","status"]).size().unstack(fill_value=0)
        .rename_axis(None, axis=1).reset_index()
    )
    for s in ["out","questionable","doubtful"]:
        if s not in counts.columns:
            counts[s] = 0
    return counts.rename(columns={
        "out": "inj_out",
        "questionable": "inj_quest",
        "doubtful": "inj_doubt"
    })

inj = scrape_injuries()

# ---------------------------------------------------------------------------
# 6. Merge everything
# ---------------------------------------------------------------------------
df = games.merge(odds, on=["home_team","away_team"], how="left")
df = df.merge(epa.add_suffix("_home"), left_on="home_team", right_on="team_home", how="left")
df = df.merge(epa.add_suffix("_away"), left_on="away_team", right_on="team_away", how="left")
df = df.merge(inj.add_suffix("_home"), left_on="home_team", right_on="team_home", how="left")
df = df.merge(inj.add_suffix("_away"), left_on="away_team", right_on="team_away", how="left")

# Fill NaNs
df.fillna(0, inplace=True)

# ---------------------------------------------------------------------------
# 7. Finalize and save
# ---------------------------------------------------------------------------
df["kickoff_et"] = df["game_date"] + " " + df["start_time"].fillna("13:00")
df["favorite"] = np.where(df["spread_fav"].astype(float) > 0, df["away_team"], df["home_team"])
cols = [
    "game_id","kickoff_et","home_team","away_team","favorite","spread_fav","total","ml_fav",
    "weather_precip","weather_temp_f","weather_breezy",
    "inj_out_home","inj_quest_home","inj_doubt_home",
    "inj_out_away","inj_quest_away","inj_doubt_away",
    "offense_epa_per_play_home","defense_epa_per_play_home",
    "offense_epa_per_play_away","defense_epa_per_play_away"
]
input_df = df[cols].copy()

out_path = Path("/mnt/data/nfl_week_inputs.csv")
input_df.to_csv(out_path, index=False)
print(f"✅ Created live NFL input file with odds, weather, EPA, and injuries → {out_path}")
input_df.head()

## Load file

In [3]:
%%python
import pandas as pd
from pathlib import Path

data = [
    # game_id, kickoff, home, away, favorite, spread, total, ml_fav, ml_dog, city, precip, tempF, breezy,
    # injury counts (dummy small values), EPA offense/defense placeholders
    ["2025W09_BAL_MIA","2025-11-02 10:00","MIA","BAL","BAL",7.5,51.5,-420,330,"Miami Gardens, FL",0,80,1,0,0,0,1,1,4,0.00,0.13,-0.01,0.11],
    ["2025W09_NYG_SF","2025-11-02 10:00","NYG","SF","SF",2.5,48.5,-150,130,"East Rutherford, NJ",0,62,0,0,0,3,3,1,2,0.00,0.13,-0.01,0.11],
    ["2025W09_HOU_DEN","2025-11-02 10:00","HOU","DEN","HOU",1.5,39.5,-125,105,"Houston, TX",0,74,0,1,0,1,2,0,1,-0.03,0.02,-0.02,0.01],
    ["2025W09_PIT_IND","2025-11-02 10:00","PIT","IND","IND",3.5,51,-170,145,"Pittsburgh, PA",0,58,0,1,0,1,2,1,2,0.02,0.12,-0.01,0.11],
    ["2025W09_CIN_CHI","2025-11-02 10:00","CIN","CHI","CHI",2.5,51.5,-150,130,"Cincinnati, OH",0,50,0,1,0,1,2,0,2,-0.01,0.11,-0.02,0.12],
    ["2025W09_GB_CAR","2025-11-02 10:00","GB","CAR","GB",12.5,43.5,-900,600,"Green Bay, WI",0,51,1,1,0,2,3,0,1,0.01,0.10,-0.02,0.13],
    ["2025W09_DET_MIN","2025-11-02 10:00","DET","MIN","DET",8.5,48.5,-420,330,"Detroit, MI",0,70,0,0,0,1,2,1,3,0.04,0.09,-0.02,0.12],
    ["2025W09_TEN_LAC","2025-11-02 10:00","TEN","LAC","LAC",10.5,43.5,-500,380,"Nashville, TN",0,53,0,1,0,1,2,0,1,0.03,0.09,-0.02,0.12],
    ["2025W09_NE_ATL","2025-11-02 10:00","NE","ATL","NE",5.5,45.5,-210,180,"Foxborough, MA",0,52,0,1,0,1,2,1,2,0.02,0.10,-0.01,0.12],
    ["2025W09_LV_JAX","2025-11-02 13:05","LV","JAX","JAX",3.5,44.5,-150,125,"Las Vegas, NV",0,72,0,1,0,1,2,0,1,0.02,0.11,-0.01,0.12],
    ["2025W09_LAR_NO","2025-11-02 13:25","LAR","NO","LAR",13.5,43.5,-1100,700,"Inglewood, CA",0,72,0,1,0,1,1,0,1,0.03,0.10,-0.02,0.12],
    ["2025W09_BUF_KC","2025-11-02 13:25","BUF","KC","KC",1.5,52.5,-130,110,"Orchard Park, NY",0,55,0,1,0,1,2,0,1,0.04,0.08,-0.02,0.12],
    ["2025W09_WAS_SEA","2025-11-02 17:20","WAS","SEA","SEA",3.5,48.5,-160,135,"Landover, MD",0,63,0,1,0,1,1,0,1,0.02,0.10,-0.01,0.11],
    ["2025W09_DAL_ARI","2025-11-03 17:15","DAL","ARI","DAL",3.5,53.5,-173,144,"Arlington, TX",0,72,0,0,0,1,1,0,1,0.03,0.10,-0.02,0.11],
]

cols = [
    "game_id","kickoff_et","home_team","away_team","favorite","spread_fav","total",
    "ml_fav","ml_dog","home_city","weather_precip","weather_temp_f","weather_breezy",
    "inj_out_fav","inj_doubt_fav","inj_quest_fav","inj_out_dog","inj_doubt_dog","inj_quest_dog",
    "off_epa_fav","def_epa_allowed_fav","off_epa_dog","def_epa_allowed_dog"
]

df = pd.DataFrame(data, columns=cols)
out_path = Path("data/nfl_week_inputs.csv")
df.to_csv(out_path, index=False)
print(f"✅ Created full input file at {out_path}")
df.head(16)

✅ Created full input file at data/nfl_week_inputs.csv


## Input CSV schema (`/mnt/data/nfl_week_inputs.csv`)

Required columns: `game_id,kickoff_et,home_team,away_team,favorite,spread_fav,total,ml_fav,ml_dog,home_city,weather_precip,weather_temp_f,weather_breezy,inj_out_fav,inj_doubt_fav,inj_quest_fav,inj_out_dog,inj_doubt_dog,inj_quest_dog,off_epa_fav,def_epa_allowed_fav,off_epa_dog,def_epa_allowed_dog`.


In [4]:
import pandas as pd
from pathlib import Path
p = Path('data/nfl_week_inputs.csv')
if not p.exists():
    demo = pd.DataFrame([
        {'game_id':'2025W09_BAL_MIA','kickoff_et':'2025-10-30 20:15','home_team':'MIA','away_team':'BAL','favorite':'BAL','spread_fav':7.5,'total':51.5,'ml_fav':-420,'ml_dog':330,'home_city':'Miami Gardens, FL','weather_precip':0,'weather_temp_f':80,'weather_breezy':1,'inj_out_fav':0,'inj_doubt_fav':0,'inj_quest_fav':0,'inj_out_dog':1,'inj_doubt_dog':1,'inj_quest_dog':4,'off_epa_fav':0.00,'def_epa_allowed_fav':0.13,'off_epa_dog':-0.01,'def_epa_allowed_dog':0.11}
    ])
    demo.to_csv(p, index=False)
    print('Template created at data/nfl_week_inputs.csv.  Add all games and rerun.')
else:
    print('Found data/nfl_week_inputs.csv — will evaluate that slate.')

Found data/nfl_week_inputs.csv — will evaluate that slate.


In [5]:
import numpy as np, math
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

def implied_prob(odds):
    return (-odds)/((-odds)+100) if odds<0 else 100/(odds+100)

def fair_probs(ml_fav, ml_dog):
    p_f = implied_prob(ml_fav); p_d = implied_prob(ml_dog)
    vig = p_f + p_d - 1.0
    return p_f/(1+vig), p_d/(1+vig)

def spread_to_prob(spread,k=0.18):
    return 1/(1+math.exp(-k*spread))

def injury_penalty(o,d,q):
    return 0.010*o + 0.006*d + 0.002*q

def monte_carlo_cover(spread, mu, sd=13.0, N=100_000, seed=20251102):
    rng = np.random.default_rng(seed)
    margins = rng.normal(mu, sd, N)
    return float(np.mean(margins>spread)), float(np.mean(margins>0)), np.percentile(margins,[5,25,50,75,95]).tolist()

def train_synth(n=20000, sd=13.0, seed=20251030):
    rng = np.random.default_rng(seed)
    spread = rng.uniform(0,12,n)
    off_f = rng.normal(0,0.10,n)
    off_d = rng.normal(0,0.10,n)
    mu = 0.75*spread + 25*off_f - 20*off_d
    margin = rng.normal(mu, sd, n)
    covered = (margin>spread).astype(int)
    X = np.column_stack([spread, off_f, off_d]); y = covered
    logit = LogisticRegression(max_iter=400).fit(X,y)
    gb = GradientBoostingClassifier(random_state=seed).fit(X,y)
    return logit, gb, roc_auc_score(y,logit.predict_proba(X)[:,1]), roc_auc_score(y,gb.predict_proba(X)[:,1])

logit_s, gb_s, aucL, aucG = train_synth()
print(f'Synthetic AUCs — Logit: {aucL:.3f}, GBM: {aucG:.3f}')

Synthetic AUCs — Logit: 0.616, GBM: 0.646


In [6]:
import pandas as pd
slate = pd.read_csv('data/nfl_week_inputs.csv')
rows = []
for _, g in slate.iterrows():
    fav = g['favorite']
    home_is_fav = (g['home_team']==fav)
    p_fair_fav, _ = fair_probs(g['ml_fav'], g['ml_dog'])
    p_spread = spread_to_prob(g['spread_fav'])
    pen_fav = injury_penalty(g['inj_out_fav'], g['inj_doubt_fav'], g['inj_quest_fav'])
    pen_dog = injury_penalty(g['inj_out_dog'], g['inj_doubt_dog'], g['inj_quest_dog'])
    home_bump = 0.015 if g['weather_precip']==0 else 0.010
    sign_home = 1 if home_is_fav else -1
    p_market_adj = max(0,min(1, p_fair_fav + sign_home*home_bump + (pen_dog - pen_fav)))
    p_spread_adj  = max(0,min(1, p_spread   + sign_home*home_bump + (pen_dog - pen_fav)))
    p_win = 0.5*(p_market_adj + p_spread_adj)

    off_diff_fav = g['off_epa_fav'] - g['def_epa_allowed_dog']
    off_diff_dog = g['off_epa_dog'] - g['def_epa_allowed_fav']
    mu_margin = float(g['spread_fav']) + 25*off_diff_fav - 20*off_diff_dog
    p_cover_mc, p_win_mc, pct = monte_carlo_cover(g['spread_fav'], mu_margin)

    X1 = [[g['spread_fav'], off_diff_fav, off_diff_dog]]
    p_cover_logit = float(logit_s.predict_proba(X1)[:,1])
    p_cover_gb    = float(gb_s.predict_proba(X1)[:,1])
    p_cover = float((p_cover_mc + 0.5*(p_cover_logit + p_cover_gb))/2)

    ats_lean = ('Lean favorite cover' if p_cover>0.53 else 'Neutral / coin‑flip' if 0.47<=p_cover<=0.53 else 'Lean dog +points')
    rows.append({
        'matchup': f"{g['away_team']} @ {g['home_team']}",
        'favorite': fav,
        'line': f"{fav} -{g['spread_fav']}",
        'total': g['total'],
        'P(win fav)': round(p_win,3),
        'P(cover fav)': round(p_cover,3),
        'ATS lean': ats_lean,
        'median margin': round(pct[2],2),
        'p05': round(pct[0],2), 'p95': round(pct[4],2),
        'vig_free_ml_win': round(p_fair_fav,3),
        'spread_win_prob': round(p_spread,3),
        'home_bump_sign': sign_home,
        'inj_pen_fav': round(pen_fav,3), 'inj_pen_dog': round(pen_dog,3),
        'off_diff_fav': round(off_diff_fav,3), 'off_diff_dog': round(off_diff_dog,3)
    })

summary = pd.DataFrame(rows)
summary

/var/folders/rn/4w3wn3_d6jv7tvd00mfp_k_w0000gn/T/ipykernel_3114/1980344491.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  p_cover_logit = float(logit_s.predict_proba(X1)[:,1])
/var/folders/rn/4w3wn3_d6jv7tvd00mfp_k_w0000gn/T/ipykernel_3114/1980344491.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  p_cover_gb    = float(gb_s.predict_proba(X1)[:,1])
/var/folders/rn/4w3wn3_d6jv7tvd00mfp_k_w0000gn/T/ipykernel_3114/1980344491.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprec

,matchup,favorite,line,total,P(win fav),P(cover fav),ATS lean,median margin,p05,p95,vig_free_ml_win,spread_win_prob,home_bump_sign,inj_pen_fav,inj_pen_dog,off_diff_fav,off_diff_dog
0,BAL @ MIA,BAL,BAL -7.5,51.5,0.794,0.465,Lean dog +points,7.59,-13.69,28.99,0.776,0.794,-1,0.000,0.024,-0.11,-0.14
1,SF @ NYG,SF,SF -2.5,48.5,0.614,0.482,Neutral / coin‑flip,2.59,-18.69,23.99,0.580,0.611,-1,0.006,0.040,-0.11,-0.14
2,DEN @ HOU,HOU,HOU -1.5,39.5,0.575,0.496,Neutral / coin‑flip,1.34,-19.94,22.74,0.532,0.567,1,0.012,0.022,-0.04,-0.04
3,IND @ PIT,IND,IND -3.5,51.0,0.633,0.492,Neutral / coin‑flip,3.89,-17.39,25.29,0.607,0.652,-1,0.012,0.030,-0.09,-0.13
4,CHI @ CIN,CHI,CHI -2.5,51.5,0.592,0.463,Lean dog +points,1.89,-19.39,23.29,0.580,0.611,-1,0.012,0.024,-0.13,-0.13
5,CAR @ GB,GB,GB -12.5,43.5,0.917,0.373,Lean dog +points,11.94,-9.34,33.34,0.863,0.905,1,0.014,0.032,-0.12,-0.12
6,MIN @ DET,DET,DET -8.5,48.5,0.844,0.472,Neutral / coin‑flip,8.74,-12.54,30.14,0.776,0.822,1,0.002,0.032,-0.08,-0.11
7,LAC @ TEN,LAC,LAC -10.5,43.5,0.829,0.457,Lean dog +points,10.49,-10.79,31.89,0.800,0.869,-1,0.012,0.022,-0.09,-0.11
8,ATL @ NE,NE,NE -5.5,45.5,0.725,0.456,Lean dog +points,5.24,-16.04,26.64,0.655,0.729,1,0.012,0.030,-0.10,-0.11
9,JAX @ LV,JAX,JAX -3.5,44.5,0.608,0.468,Lean dog +points,3.44,-17.84,24.84,0.574,0.652,-1,0.012,0.022,-0.10,-0.12


In [7]:
out_csv = Path('data/nfl_week_summary.csv')
summary.to_csv(out_csv, index=False)
print(f'Saved weekly summary to {out_csv}')

Saved weekly summary to data/nfl_week_summary.csv
